In [ ]:
import math
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
from graphviz import Digraph

#construct nodes and edges
def construct(node, nodes, edges):
    if node not in nodes:
        nodes.add(node)
        for child in node.previous:
            edges.add((child, node))
            construct(child, nodes, edges)

#calls construct for the first time
def dfs(root):
    nodes = set()
    edges = set()
    construct(root, nodes, edges)
    return nodes, edges

#constructs the graph
def graph(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    nodes, edges = dfs(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.name, n.value, n.grad), shape='record')
        if n.operator:
          dot.node(name = uid + n.operator, label = n.operator)
          dot.edge(uid + n.operator, uid)

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2.operator)

    return dot

In [ ]:
class Variable:

    def __init__(self, value, relation=(), operator='', name=''):
        self.value = value
        self.grad = 0.0
        self.previous = set(relation)
        self.operator = operator
        self.name = name
        self.back = lambda: None        # define an empty function for now

    def __repr__(self):
        if(self.name != ''):
            return f"Variable(value={self.value}, name='{self.name}')"
        return f"Variable(value={self.value})"

    def __add__(self, next):
        next = next if isinstance(next, Variable) else Variable(next)
        out = Variable(self.value + next.value, (self, next), '+')

        def back():
            self.grad += 1.0 * out.grad
            next.grad += 1.0 * out.grad
        out.back = back
        return out

    def __sub__(self, next):
        return self + (-next)

    def __rsub__(self, next):
        return next + (-self)

    def __neg__(self):
        return self * -1

    def __radd__(self, next):
        return self + next

    def __truediv__(self, next):
        return self * next**-1

    def __rtruediv(self, next):
        return next * self**-1

    def __pow__(self, next):
        output = Variable(self.value ** next, (self, ), f'**{next}')

        def back():
            self.grad += next * (self.value ** (next - 1)) * output.grad
        output.back = back

        return output

    def __mul__(self, next):
        next = next if isinstance(next, Variable) else Variable(next)
        out = Variable(self.value * next.value, (self, next), '*')

        def back():
            self.grad += next.value * out.grad
            next.grad += self.value * out.grad
        out.back = back
        
        return out

    def log(self):
        epsilon = 1e-7
        if self.value <= 0:
            raise ValueError("Cannot take the logarithm of a non-positive number")
        z = self.value
        out = Variable(math.log(z + epsilon), (self, ), 'log')
    
        def back():
            self.grad += (1/(self.value + epsilon)) * out.grad
    
        out.back = back
    
        return out

    def __rmul__(self, next):
        return self * next

    def sigmoid(self):
        z = self.value
        temp = (1/(1+math.exp(-1*z)))
        output = Variable(temp, (self, ), 'sig')

        def back():
            self.grad += temp * (1 - temp) * output.grad
        output.back = back
        
        return output

    def tanh(self):
        z = self.value
        temp = (math.exp(2*z) - 1)/(math.exp(2*z) + 1)
        output = Variable(temp, (self, ), 'tanh')

        def back():
            self.grad += (1 - temp**2) * output.grad

        output.back = back

        return output

    def exp(self):
        z = self.value
        out = Variable(math.exp(z), (self, ), 'exp')

        def back():
            self.grad += out.value * out.grad

        out.back = back

        return out

    def backward(self):
        def topological_sort(node):
            stack = [node]
            visited = set()
            list_in_order = []
            while stack:
                node = stack[-1]
                if node not in visited:
                    visited.add(node)
                    stack.extend(reversed(list(node.previous)))
                else:
                    list_in_order.append(stack.pop())
            return list_in_order

        list_in_order = []
        list_in_order = topological_sort(self)

        self.grad = 1.0
        list_in_order.reverse()
        for node in list_in_order:
            node.back()


In [ ]:
class Neuron:

    def __init__(self, number_of_inputs):
        self.weight = [Variable(random.uniform(-1, 1)) for _ in range(number_of_inputs)]
        self.bias = Variable(random.uniform(-1, 1))

    def __call__(self, input):
        # a = summation (wi*xi) + b
        
        # weighted_input = (inputi*weighti for inputi, weighti in zip(input, self.weight))
        activation = sum((weighti*inputi for inputi, weighti in zip(input, self.weight)), self.bias)
        output = activation.tanh()
        return output

    def definition(self):
        return self.weight + [self.bias]

class Layer:

    def __init__(self, number_of_input, number_of_output):
        self.neurons = [Neuron(number_of_input) for _ in range(number_of_output)]

    def __call__(self, input):
        output = [temp(input) for temp in self.neurons]
        if(len(output) == 1):
            return output[0]
        return output

    def definition(self):
        output = []
        for neuron in self.neurons:
            for temp in neuron.definition():
                output.append(temp)
        return output

class Network:

    def __init__(self, number_of_inputs, number_of_outputs):
        temp = [number_of_inputs] + number_of_outputs
        self.layers = [Layer(temp[i], temp[i+1]) for i in range(len(number_of_outputs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def definition(self):
        output = []
        for layer in self.layers:
            for temp in layer.definition():
                output.append(temp)
        return output

In [ ]:
df = pd.read_csv(r"banana.csv")
df

In [ ]:
X = df[['At1', 'At2']].values
y = df['Class'].values

# Plot the dataset
plt.figure(figsize=(10, 6))
plt.scatter(X[y == -1][:, 0], X[y == -1][:, 1], color='red', label='Class -1')
plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], color='blue', label='Class 1')
plt.title('Banana Dataset')
plt.xlabel('At1')
plt.ylabel('At2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .076)

In [ ]:
net = Network(2, [16, 16, 1])

In [ ]:
# for temp in y_train:
#     temp = (temp+1)/2

# for iteration in range(20):

#     # forward pass -------------------------------------------------------------
#     y_predictions = [net(temp) for temp in X_train]
#     loss = sum(-y_desired * y_prediction.log() - (1-y_desired) * (1-y_prediction).log() for y_desired, y_prediction in zip(y_train, y_predictions)) / len(X_train)
#     # --------------------------------------------------------------------------

#     # backward pass ------------------------------------------------------------
#     for temp in net.definition():
#         temp.grad = 0.0

#     loss.backward()
#     # --------------------------------------------------------------------------

#     # update -------------------------------------------------------------------
#     for temp in net.definition():
#         temp.value += -0.1 * temp.grad
#     # --------------------------------------------------------------------------

#     print(f"Iteration number {iteration}: {loss.value}")

In [ ]:
# list(zip(y_predictions, (y_train+1)/2))

# Consider 40 epochs

In [ ]:
losses = []
# create log file for loss and epoch number
file = open("losses.txt", "w")

for iteration in range(80):
    # forward pass -------------------------------------------------------------
    y_predictions = [net(temp) for temp in X_train]
    loss = sum((y_prediction - y_desired)**2 for y_desired, y_prediction in zip(y_train, y_predictions)) / (2*len(X_train))
    # --------------------------------------------------------------------------
    
    # backward pass ------------------------------------------------------------
    for temp in net.definition():
        temp.grad = 0.0
    
    loss.backward()
    # --------------------------------------------------------------------------
    
    # update -------------------------------------------------------------------
    for temp in net.definition():
        temp.value += -0.1 * temp.grad
    # --------------------------------------------------------------------------
    
    # Store loss for plotting
    losses.append(loss.value)
    # update log file with loss and epoch number
    file.write(f"Iteration number {iteration}: {loss.value}\n")
    
    print(f"Iteration number {iteration}: {loss.value}")

In [ ]:
# Plot epoch graph after all iterations
plt.plot(range(80), losses, color='blue')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss per Iteration')
plt.show()

In [ ]:
list(zip(y_predictions, y_train))

In [ ]:
y_test_pred = [net(temp).value for temp in X_test]

In [ ]:
def compute_accuracy(y_pred, y_true):
    # Convert the predicted probabilities to 1 or -1 based on a threshold of 0
    predictions = np.where(y_pred > 0.0, 1, -1)
    # Calculate the accuracy as the mean of correct predictions
    accuracy = np.mean(predictions == y_true)
    return accuracy

# Convert y_test_pred to a numpy array if it's not already
y_test_pred = np.array(y_test_pred)

# Compute the test set accuracy
test_accuracy = compute_accuracy(y_test_pred, y_test)

print(f"Test set accuracy: {test_accuracy}")

In [ ]:
def compute_accuracy(y_pred, y_true):
    # Convert the predicted probabilities to 1 or -1 based on a threshold of 0.3 and -0.3
    predictions = np.where(y_pred > 0.3, 1, np.where(y_pred < -0.3, -1, 0))
    # Calculate the accuracy as the mean of correct predictions
    accuracy = np.mean(predictions == y_true)
    return accuracy

# Convert y_test_pred to a numpy array if it's not already
y_test_pred = np.array(y_test_pred)

# Compute the test set accuracy
test_accuracy = compute_accuracy(y_test_pred, y_test)

print(f"Test set accuracy: {test_accuracy}")